In [5]:
!pip install datasets
!pip install evaluate


   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 480.6/480.6 kB 22.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 116.3/116.3 kB 12.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 179.3/179.3 kB 18.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 134.8/134.8 kB 14.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 194.1/194.1 kB 17.3 MB/s eta 0:00:00
  Attempting uninstall: fsspec
    Found existing installation: fsspec 2024.10.0
    Uninstalling fsspec-2024.10.0:
      Successfully uninstalled fsspec-2024.10.0
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
gcsfs 2024.10.0 requires fsspec==2024.10.0, but you have fsspec 2024.9.0 which is incompatible.
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 84.0/84.0 kB 5.9 MB/s eta 0:00:00


In [2]:
import pandas as pd

train_csv_path = "/content/train.csv"
train_data = pd.read_csv(train_csv_path)



print("Training Dataset:")
print(train_data.head())




Training Dataset:
   SampleID                                         Discussion  Category
0         1  Without sitting down and doing it manually, yo...    Sports
1         2               All your Search ends with this link.      STEM
2         3  No, the program you're using is made to be com...      STEM
3         4  Mike Woicik\n\nThe correct answer is: Mike Woi...    Sports
4         5  No, but not because of why you might think. Wh...  Politics


In [3]:
from transformers import AutoTokenizer, AutoModelForSequenceClassification

tokenizer = AutoTokenizer.from_pretrained("distilbert/distilbert-base-uncased-finetuned-sst-2-english")


/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


tokenizer_config.json:   0%|          | 0.00/48.0 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/629 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/232k [00:00<?, ?B/s]

In [6]:
from transformers import BertTokenizer, BertForSequenceClassification, Trainer, TrainingArguments,EarlyStoppingCallback
from sklearn.model_selection import train_test_split
from datasets import Dataset
import pandas as pd




train_data = train_data.dropna(subset=["Discussion", "Category"])
category_mapping = {category: idx for idx, category in enumerate(train_data["Category"].unique())}
train_data["Category"] = train_data["Category"].map(category_mapping)

train_texts, test_texts, train_labels, test_labels = train_test_split(
    train_data["Discussion"].tolist(),
    train_data["Category"].tolist(),
    test_size=0.2,
    random_state=42
)

train_dataset = Dataset.from_dict({"text": train_texts, "label": train_labels})
test_dataset = Dataset.from_dict({"text": test_texts, "label": test_labels})

model_name = "bert-base-uncased"
tokenizer = BertTokenizer.from_pretrained(model_name)
model = BertForSequenceClassification.from_pretrained(model_name, num_labels=len(category_mapping))

def preprocess_data(examples):
    return tokenizer(examples["text"], padding="max_length", truncation=True, max_length=128)

train_dataset = train_dataset.map(preprocess_data, batched=True)
test_dataset = test_dataset.map(preprocess_data, batched=True)

train_dataset = train_dataset.remove_columns(["text"])
test_dataset = test_dataset.remove_columns(["text"])

training_args = TrainingArguments(
    output_dir="./results",
    evaluation_strategy="epoch",
    learning_rate=2e-5,
    per_device_train_batch_size=8,
    per_device_eval_batch_size=8,
    num_train_epochs=4,
    weight_decay=0.5,
    logging_dir="./logs",
    logging_steps=10,
    save_strategy="epoch",
    load_best_model_at_end=True,
    metric_for_best_model="accuracy",
    fp16=True  # Enable mixed precision for faster training on GPU
)

from sklearn.metrics import accuracy_score, precision_recall_fscore_support
def compute_metrics(eval_pred):
    logits, labels = eval_pred
    predictions = logits.argmax(axis=1)
    precision, recall, f1, _ = precision_recall_fscore_support(labels, predictions, average="weighted")
    acc = accuracy_score(labels, predictions)
    return {"accuracy": acc, "f1": f1, "precision": precision, "recall": recall}

trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=train_dataset,
    eval_dataset=test_dataset,
    tokenizer=tokenizer,
    compute_metrics=compute_metrics,
    callbacks=[EarlyStoppingCallback(early_stopping_patience=2)]
)
trainer.train()

metrics = trainer.evaluate()
print("Evaluation Metrics:", metrics)


   SampleID                                         Discussion  Category
0         1  Without sitting down and doing it manually, yo...    Sports
1         2               All your Search ends with this link.      STEM
2         3  No, the program you're using is made to be com...      STEM
3         4  Mike Woicik\n\nThe correct answer is: Mike Woi...    Sports
4         5  No, but not because of why you might think. Wh...  Politics


tokenizer_config.json:   0%|          | 0.00/48.0 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/232k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/466k [00:00<?, ?B/s]

config.json:   0%|          | 0.00/570 [00:00<?, ?B/s]

...

wandb: Logging into wandb.ai. (Learn how to deploy a W&B server locally: https://wandb.me/wandb-server)
wandb: You can find your API key in your browser here: https://wandb.ai/authorize
wandb: Paste an API key from your profile and hit enter, or press ctrl+c to quit:

 ··········


wandb: Appending key for api.wandb.ai to your netrc file: /root/.netrc


Epoch,Training Loss,Validation Loss,Accuracy,F1,Precision,Recall
1,0.658500,0.680825,0.761460,0.761477,0.763842,0.761460
2,0.394500,0.710628,0.760041,0.760466,0.763933,0.760041
3,0.426000,0.969780,0.756389,0.757227,0.759957,0.756389


Evaluation Metrics: {'eval_loss': 0.6808252334594727, 'eval_accuracy': 0.7614604462474645, 'eval_f1': 0.7614765113871628, 'eval_precision': 0.7638415973064306, 'eval_recall': 0.7614604462474645, 'eval_runtime': 12.4777, 'eval_samples_per_second': 395.104, 'eval_steps_per_second': 49.448, 'epoch': 3.0}


In [12]:
from transformers import BertTokenizer, BertForSequenceClassification, Trainer, TrainingArguments
from datasets import Dataset
import pandas as pd


train_data = train_data.dropna(subset=["Discussion", "Category"])

category_mapping = {cat: i for i, cat in enumerate(train_data["Category"].unique())}
idx_to_category = {v: k for k, v in category_mapping.items()}

train_data["label"] = train_data["Category"].map(category_mapping)

train_dataset = Dataset.from_pandas(train_data[["Discussion", "label"]])

test_csv_path = "/content/test.csv"
test_data = pd.read_csv(test_csv_path)
test_data = test_data.dropna(subset=["Discussion"])

test_dataset = Dataset.from_pandas(test_data[["Discussion"]])
model_name = "bert-base-uncased"
tokenizer = BertTokenizer.from_pretrained(model_name)
model = BertForSequenceClassification.from_pretrained(model_name, num_labels=len(category_mapping))

def preprocess_data(examples):
    return tokenizer(examples["Discussion"], truncation=True, padding="max_length", max_length=128)

train_dataset = train_dataset.map(preprocess_data, batched=True)
test_dataset = test_dataset.map(preprocess_data, batched=True)

train_dataset = train_dataset.remove_columns(["Discussion"])
test_dataset = test_dataset.remove_columns(["Discussion"])

train_dataset.set_format("torch")
test_dataset.set_format("torch")

training_args = TrainingArguments(
    output_dir="./results",
    evaluation_strategy="no",
    learning_rate=2e-5,
    per_device_train_batch_size=8,
    per_device_eval_batch_size=8,
    num_train_epochs=4,
    weight_decay=0.5,
    logging_dir="./logs",
    logging_steps=10,
    save_strategy="epoch",
    fp16=True,
)

trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=train_dataset,
    tokenizer=tokenizer
)

trainer.train()

predictions = trainer.predict(test_dataset)
pred_logits = predictions.predictions
pred_class_indices = pred_logits.argmax(axis=1)

pred_categories = [idx_to_category[idx] for idx in pred_class_indices]

test_data["Predicted_Category"] = pred_categories

test_data.to_csv("/content/test_with_predictions.csv", index=False)
print("Predicted categories saved to /content/test_with_predictions.csv")


Some weights of BertForSequenceClassification were not initialized from the model checkpoint at bert-base-uncased and are newly initialized: ['classifier.bias', 'classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


Map:   0%|          | 0/24646 [00:00<?, ? examples/s]

Map:   0%|          | 0/10557 [00:00<?, ? examples/s]

/usr/local/lib/python3.10/dist-packages/transformers/training_args.py:1568: FutureWarning: `evaluation_strategy` is deprecated and will be removed in version 4.46 of 🤗 Transformers. Use `eval_strategy` instead
  warnings.warn(
<ipython-input-12-70181edbb308>:62: FutureWarning: `tokenizer` is deprecated and will be removed in version 5.0.0 for `Trainer.__init__`. Use `processing_class` instead.
  trainer = Trainer(


Step,Training Loss
10,1.642400
20,1.543500
30,1.594300
40,1.573300
50,1.546600
60,1.449100
70,1.349800
80,1.224400
90,1.282500
100,1.103900


Predicted categories saved to /content/test_with_predictions.csv
